In [2]:
!pip3 install flask
!pip3 install pandas
!pip3 install scikit-learn
!pip3 install joblib
!pip3 install flask-ngrok

In [3]:
import pandas as pd
import numpy as np
from flask import Flask, request, jsonify, render_template
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix
import joblib
from flask_ngrok import run_with_ngrok # Import the flask_ngrok library


In [4]:
# Load the dataset
# Replace 'path_to_dataset' with the path where you have downloaded 'XSS_dataset.csv'
data = pd.read_csv('XSS_dataset.csv')

# Splitting data into features and target
X = data['Sentence']
y = data['Label']

print(X)



# Vectorizing the text data
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(X)

# Splitting data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Training the model
model = SVC(kernel='linear')
model.fit(X_train, y_train)

# Save the model
joblib.dump(model, 'xss_detection_model.joblib')


0        <li><a href="/wiki/File:Socrates.png" class="i...
1                     <tt onmouseover="alert(1)">test</tt>
2        \t </span> <span class="reference-text">Steeri...
3        \t </span> <span class="reference-text"><cite ...
4        \t </span>. <a href="/wiki/Digital_object_iden...
                               ...                        
13681               <img onpointerenter=alert(1)>XSS</img>
13682    <source onbeforepaste="alert(1)" contenteditab...
13683    <div draggable="true" contenteditable>drag me<...
13684    <li><cite id="CITEREFDomingos2015" class="cita...
13685                                           \t </span>
Name: Sentence, Length: 13686, dtype: object


['xss_detection_model.joblib']

In [5]:
!export FLASK_ENV=development
import pdb

app = Flask(__name__)

# Load the model
model = joblib.load('xss_detection_model.joblib')

# Add the run_with_ngrok(app) function to tunnel the app to a public URL

@app.route('/')
def home():
    return render_template('index.html')

@app.route('/predict', methods=['POST'])
def predict():
    if request.method == 'POST':
        text = request.form['text']
        text_vectorized = vectorizer.transform([text])
        # pdb.set_trace()  # Set a breakpoint here
        prediction = model.predict(text_vectorized)
        if prediction[0] == 1:
            result = 'XSS Attack Detected!'
        else:
            print(prediction[0])
            result = 'No XSS Attack Detected'
        return render_template('result.html', result=result)
        
if __name__ == '__main__':
    # Use the app.run(debug=True) method to run the app in debug mode
    app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [29/Feb/2024 02:10:38] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [29/Feb/2024 02:10:41] "POST /predict HTTP/1.1" 200 -


0


127.0.0.1 - - [29/Feb/2024 02:10:42] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [29/Feb/2024 02:11:14] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [29/Feb/2024 02:11:15] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [29/Feb/2024 02:11:18] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [29/Feb/2024 02:11:20] "GET / HTTP/1.1" 200 -
